# 1. Junos Device Connection Setup 

In [77]:
from jnpr.junos import Device

connection = {'user':'lab', 'password':'lab123', 'host':'192.168.1.11'}

dev = Device (**connection)

In [76]:
dev.open()

ConnectTimeoutError: ConnectTimeoutError(192.168.1.11)

## 1.1. Displaying Device Facts

In [18]:
dev.facts

{'2RE': False,
 'HOME': '/var/home/lab',
 'RE0': {'last_reboot_reason': 'Router rebooted after a normal shutdown.',
  'mastership_state': 'master',
  'model': 'EX2200-C-12P-2G, POE',
  'status': 'Absent',
  'up_time': '2 hours, 26 minutes, 5 seconds'},
 'domain': None,
 'fqdn': '',
 'hostname': '',
 'ifd_style': 'SWITCH',
 'master': 'RE0',
 'model': 'EX2200-C-12P-2G, POE+',
 'personality': 'SWITCH',
 'serialnumber': 'GR0214472053',
 'switch_style': 'VLAN',
 'vc_capable': True,
 'vc_mode': 'Enabled',
 'version': '12.3R6.6',
 'version_RE0': '12.3R6.6',
 'version_info': junos.version_info(major=(12, 3), type=R, minor=6, build=6)}

## 1.2. Executing Operational Commands 

### 1.2.1. Executing an Operational CLI command (debug only - not recommended)

In [19]:
chassis = dev.cli('show chassis hardware', warning=False) 
print(chassis)


Hardware inventory:
Item             Version  Part number  Serial number     Description
Chassis                                GR0214472053      EX2200-C-12P-2G, POE+
Routing Engine 0 REV 10   650-036547   GR0214472053      EX2200-C-12P-2G, POE+
FPC 0            REV 10   650-036547   GR0214472053      EX2200-C-12P-2G, POE+
  CPU                     BUILTIN      BUILTIN           FPC CPU
  PIC 0                   BUILTIN      BUILTIN           12x 10/100/1000 Base-T
  PIC 1          REV 10   650-036547   GR0214472053      2x (10/100/1000 Base-T or GE SFP)
Power Supply 0                                           PS 180W AC



### 1.2.2. Extracting XML RPC Command (via CLI - then using the native xml_rpc command). 

In [58]:
from lxml import etree 
from jxmlease import *
myparser = Parser()

chassis = dev.cli('show chassis hardware | display xml rpc', warning=False) 
print etree.tostring(chassis)

<get-chassis-inventory>
</get-chassis-inventory>



In [59]:
root = myparser( etree.tostring(dev.rpc.get_chassis_inventory())  )
print root.emit_xml()

<?xml version="1.0" encoding="utf-8"?>
<chassis-inventory>
    <chassis style="inventory">
        <name>Chassis</name>
        <serial-number>GR0214472053</serial-number>
        <description>EX2200-C-12P-2G, POE+</description>
        <chassis-module>
            <name>Routing Engine 0</name>
            <version>REV 10</version>
            <part-number>650-036547</part-number>
            <serial-number>GR0214472053</serial-number>
            <description>EX2200-C-12P-2G, POE+</description>
            <clei-code>CMMH610BRA</clei-code>
            <model-number>EX2200-C-12P-2G</model-number>
        </chassis-module>
        <chassis-module>
            <name>FPC 0</name>
            <version>REV 10</version>
            <part-number>650-036547</part-number>
            <serial-number>GR0214472053</serial-number>
            <description>EX2200-C-12P-2G, POE+</description>
            <clei-code>CMMH610BRA</clei-code>
            <model-number>EX2200-C-12P-2G</model-number>
      

## 1.2.3. Converting CLI commands and calling Lambda Function! 

In [61]:
cliCommand = 'show interface ge-* terse'
xml_rpc = dev.display_xml_rpc(cliCommand)
print etree.tostring(xml_rpc).strip()

<get-interface-information>
    <terse/>
    <interface-name>ge-*</interface-name>
</get-interface-information>


In [62]:
# You can enjoy entering the xml_rpc manually 

# intTerse = dev.rpc.get_interface_information(terse=True, interface_name="ge-*")
# presult = parse(etree.tostring(intTerse))
# print presult.emit_xml()

# but better to use: 
rpcMethod = getattr(dev, 'rpc')
result = rpcMethod(xml_rpc)
presult = parse(etree.tostring(result))
print presult.emit_xml()

<?xml version="1.0" encoding="utf-8"?>
<interface-information style="terse">
    <physical-interface>
        <name>ge-0/0/0</name>
        <admin-status>up</admin-status>
        <oper-status>down</oper-status>
        <logical-interface>
            <name>ge-0/0/0.0</name>
            <admin-status>up</admin-status>
            <oper-status>down</oper-status>
            <filter-information></filter-information>
            <address-family>
                <address-family-name>eth-switch</address-family-name>
            </address-family>
        </logical-interface>
    </physical-interface>
    <physical-interface>
        <name>ge-0/0/1</name>
        <admin-status>up</admin-status>
        <oper-status>down</oper-status>
        <logical-interface>
            <name>ge-0/0/1.0</name>
            <admin-status>up</admin-status>
            <oper-status>down</oper-status>
            <filter-information></filter-information>
            <address-family>
                <address-fam

## 1.3. Using Tables / Views 

In [37]:
from jnpr.junos.op.phyport import PhyPortTable 

ports = PhyPortTable(dev).get()
for port in ports: 
    print ("Port: %s, Status: %s, last Flap: %s") % (port.key, port.oper, port.flapped)

Port: ge-0/0/0, Status: down, last Flap: Never
Port: ge-0/0/1, Status: down, last Flap: Never
Port: ge-0/0/2, Status: down, last Flap: Never
Port: ge-0/0/3, Status: down, last Flap: Never
Port: ge-0/0/4, Status: down, last Flap: Never
Port: ge-0/0/5, Status: down, last Flap: Never
Port: ge-0/0/6, Status: down, last Flap: Never
Port: ge-0/0/7, Status: down, last Flap: Never
Port: ge-0/0/8, Status: down, last Flap: Never
Port: ge-0/0/9, Status: down, last Flap: Never
Port: ge-0/0/10, Status: up, last Flap: 2014-03-13 11:51:09 UTC (01:27:06 ago)
Port: ge-0/0/11, Status: down, last Flap: Never
Port: ge-0/1/0, Status: down, last Flap: Never
Port: ge-0/1/1, Status: down, last Flap: Never


In [45]:
print ports[10].to_json()

{"ge-0/0/10": {"oper": "up", "macaddr": "f0:1c:2d:bf:0c:cd", "description": null, "admin": "up", "link_mode": null, "mtu": 1514, "flapped": "2014-03-13 11:51:09 UTC (01:27:06 ago)", "speed": "Auto"}}


In [54]:
print ports[10].key

ge-0/0/10


In [53]:
ports[10].items()

[('oper', 'up'),
 ('macaddr', 'f0:1c:2d:bf:0c:cd'),
 ('description', None),
 ('link_mode', None),
 ('admin', 'up'),
 ('flapped', '2014-03-13 11:51:09 UTC (01:27:06 ago)'),
 ('speed', 'Auto'),
 ('mtu', 1514)]

# 2.1. Configuration Commands

In [56]:
config = dev.rpc.get_config() # default is xml format 

In [74]:
resp = myparser( etree.tostring(config))
print resp.emit_xml()

<?xml version="1.0" encoding="utf-8"?>
<configuration changed-seconds="1394711599" changed-localtime="2014-03-13 11:53:19 UTC">
    <version>12.3R6.6</version>
    <system>
        <auto-snapshot></auto-snapshot>
        <root-authentication>
            <encrypted-password>$1$MyXsnYTu$3e.uFL01TOV1ti.EVwpsR1</encrypted-password>
        </root-authentication>
        <login>
            <user>
                <name>lab</name>
                <uid>2000</uid>
                <class>super-user</class>
                <authentication>
                    <encrypted-password>$1$3YrKCIBC$harGVsRAN3BiJb4ownrT10</encrypted-password>
                </authentication>
            </user>
        </login>
        <services>
            <ftp></ftp>
            <ssh></ssh>
            <telnet></telnet>
            <netconf>
                <ssh></ssh>
            </netconf>
            <web-management>
                <http></http>
            </web-management>
            <dhcp>
                <t

In [73]:
config.findtext("version")

'12.3R6.6'

In [68]:
config.findtext("system/host-name")

In [72]:
login = config.find('./system/login') 

names = login.findall('user')

for name in names: 
    print "User: %s, Class=%s" % (name.findtext('name'), name.findtext('class')) 

User: lab, Class=super-user


In [71]:
presult = parse(etree.tostring(login))

print presult.emit_xml()

<?xml version="1.0" encoding="utf-8"?>
<login>
    <user>
        <name>lab</name>
        <uid>2000</uid>
        <class>super-user</class>
        <authentication>
            <encrypted-password>$1$3YrKCIBC$harGVsRAN3BiJb4ownrT10</encrypted-password>
        </authentication>
    </user>
</login>
